In [1]:
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
from pyspark.sql.functions import *

In [2]:

spark = SparkSession.builder.appName('loadRawData').getOrCreate()
file = '/Users/ybatash/PycharmProjects/jupyter/tau-network-science/data/raw_data/flightlist_*.csv.gz'
df = spark.read.csv(path=file, header=True)
df = df.withColumn("filename", input_file_name())
df.createOrReplaceTempView('T')
df.printSchema()


In [16]:
spark.sql("""
select 
        count(*) as row_count,
        sum( case when callsign     is not null then 1 else 0 end)/count(*) * 100 as nn_count_callsign     ,
        sum( case when number       is not null then 1 else 0 end)/count(*) * 100 as nn_count_number       ,
        sum( case when icao24       is not null then 1 else 0 end)/count(*) * 100 as nn_count_icao24       ,
        sum( case when registration is not null then 1 else 0 end)/count(*) * 100 as nn_count_registration ,
        sum( case when typecode     is not null then 1 else 0 end)/count(*) * 100 as nn_count_typecode     ,
        sum( case when origin       is not null then 1 else 0 end)/count(*) * 100 as nn_count_origin       ,
        sum( case when destination  is not null then 1 else 0 end)/count(*) * 100 as nn_count_destination  ,

       sum( case
                  when origin       is not null and
                       destination  is not null
                    then 1 else 0 end)/count(*) * 100 as nn_count_origin_destination       ,


        sum( case when firstseen    is not null then 1 else 0 end)/count(*) * 100 as nn_count_firstseen    ,
        sum( case when lastseen     is not null then 1 else 0 end)/count(*) * 100 as nn_count_lastseen     ,
        sum( case when day          is not null then 1 else 0 end)/count(*) * 100 as nn_count_day          ,
        sum( case when latitude_1   is not null then 1 else 0 end)/count(*) * 100 as nn_count_latitude_1   ,
        sum( case when longitude_1  is not null then 1 else 0 end)/count(*) * 100 as nn_count_longitude_1  ,
        sum( case when altitude_1   is not null then 1 else 0 end)/count(*) * 100 as nn_count_altitude_1   ,
        sum( case when latitude_2   is not null then 1 else 0 end)/count(*) * 100 as nn_count_latitude_2   ,
        sum( case when longitude_2  is not null then 1 else 0 end)/count(*) * 100 as nn_count_longitude_2  ,
        sum( case when altitude_2   is not null then 1 else 0 end)/count(*) * 100 as nn_count_altitude_2
from T
where origin is not null and destination  is not null
""").show(10, False)


+---------+-----------------+------------------+----------------+---------------------+-----------------+---------------+--------------------+---------------------------+------------------+-----------------+------------+-------------------+--------------------+-------------------+-------------------+--------------------+-------------------+
|row_count|nn_count_callsign|nn_count_number   |nn_count_icao24 |nn_count_registration|nn_count_typecode|nn_count_origin|nn_count_destination|nn_count_origin_destination|nn_count_firstseen|nn_count_lastseen|nn_count_day|nn_count_latitude_1|nn_count_longitude_1|nn_count_altitude_1|nn_count_latitude_2|nn_count_longitude_2|nn_count_altitude_2|
+---------+-----------------+------------------+----------------+---------------------+-----------------+---------------+--------------------+---------------------------+------------------+-----------------+------------+-------------------+--------------------+-------------------+-------------------+-------------

In [20]:
spark.sql("""
select 
        filename,
        count(*) as row_count,
        sum( case when callsign     is not null then 1 else 0 end)/count(*) * 100 as nn_count_callsign     ,
        sum( case when number       is not null then 1 else 0 end)/count(*) * 100 as nn_count_number       ,
        sum( case when icao24       is not null then 1 else 0 end)/count(*) * 100 as nn_count_icao24       ,
        sum( case when registration is not null then 1 else 0 end)/count(*) * 100 as nn_count_registration ,
        sum( case when typecode     is not null then 1 else 0 end)/count(*) * 100 as nn_count_typecode     ,
        sum( case when origin       is not null then 1 else 0 end)/count(*) * 100 as nn_count_origin       ,
        sum( case when destination  is not null then 1 else 0 end)/count(*) * 100 as nn_count_destination  ,

       sum( case
                  when origin       is not null and
                       destination  is not null
                    then 1 else 0 end)/count(*) * 100 as nn_count_origin_destination       ,


        sum( case when firstseen    is not null then 1 else 0 end)/count(*) * 100 as nn_count_firstseen    ,
        sum( case when lastseen     is not null then 1 else 0 end)/count(*) * 100 as nn_count_lastseen     ,
        sum( case when day          is not null then 1 else 0 end)/count(*) * 100 as nn_count_day          ,
        sum( case when latitude_1   is not null then 1 else 0 end)/count(*) * 100 as nn_count_latitude_1   ,
        sum( case when longitude_1  is not null then 1 else 0 end)/count(*) * 100 as nn_count_longitude_1  ,
        sum( case when altitude_1   is not null then 1 else 0 end)/count(*) * 100 as nn_count_altitude_1   ,
        sum( case when latitude_2   is not null then 1 else 0 end)/count(*) * 100 as nn_count_latitude_2   ,
        sum( case when longitude_2  is not null then 1 else 0 end)/count(*) * 100 as nn_count_longitude_2  ,
        sum( case when altitude_2   is not null then 1 else 0 end)/count(*) * 100 as nn_count_altitude_2
from T
where origin is not null and destination  is not null
group by 1
""").show(10, False)


+---------------------------------------------------------------------------------------------------------+---------+-----------------+------------------+-----------------+---------------------+------------------+---------------+--------------------+---------------------------+------------------+-----------------+------------+-------------------+--------------------+-------------------+-------------------+--------------------+-------------------+
|filename                                                                                                 |row_count|nn_count_callsign|nn_count_number   |nn_count_icao24  |nn_count_registration|nn_count_typecode |nn_count_origin|nn_count_destination|nn_count_origin_destination|nn_count_firstseen|nn_count_lastseen|nn_count_day|nn_count_latitude_1|nn_count_longitude_1|nn_count_altitude_1|nn_count_latitude_2|nn_count_longitude_2|nn_count_altitude_2|
+-------------------------------------------------------------------------------------------------

In [29]:
res_df = spark.sql("""
select origin                   as origin_airport,
       destination              as destination_airport,
       count(*)                 as flights_count 
from   T       
where   1=1
        and origin is not null and destination  is not null
        and filename like 'file:///Users/ybatash/PycharmProjects/jupyter/tau-network-science/data/raw_data/flightlist_2019_%.csv.gz'
group by 1,2        
""")


In [31]:
res_df.repartition(1).write.format("com.databricks.spark.csv").option("header", "true").save("mydata.csv")


In [24]:
spark.sql("""
select origin                   as origin_airport,
       destination              as destination_airport,
       count(*)                 as flights_count,
       count(distinct callsign) as distinct_callsign_count,
       count(distinct icao24)   as distinct_icao24_count,
       min(firstseen)           as min_firstseen,
       max(firstseen)           as max_firstseen 
from   T       
where   1=1
        and origin is not null and destination  is not null
        and filename = 'file:///Users/ybatash/PycharmProjects/jupyter/tau-network-science/data/raw_data/flightlist_2019_01.csv.gz'
group by 1,2        
""").show(10, False)


+--------------+-------------------+-------------+-----------------------+---------------------+-------------------------+-------------------------+
|origin_airport|destination_airport|flights_count|distinct_callsign_count|distinct_icao24_count|min_firstseen            |max_firstseen            |
+--------------+-------------------+-------------+-----------------------+---------------------+-------------------------+-------------------------+
|01FA          |KLKR               |2            |2                      |2                    |2019-01-06 22:34:07+00:00|2019-01-09 15:36:47+00:00|
|05IN          |KORD               |1            |1                      |1                    |2019-01-31 22:13:31+00:00|2019-01-31 22:13:31+00:00|
|05MD          |KGAI               |1            |1                      |1                    |2019-01-31 20:34:36+00:00|2019-01-31 20:34:36+00:00|
|06NC          |94NC               |1            |1                      |1                    |2019-01-08